<a href="https://colab.research.google.com/github/YangxuanWu/Python/blob/master/Project2020Fall/%E7%BB%83%E4%B9%A0%EF%BC%88project%EF%BC%89/project_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install git+https://github.com/thu-ml/zhusuan.git
%pip install git+https://github.com/haowen-xu/tfsnippet.git@v0.2.0-alpha1

  Cloning https://github.com/thu-ml/zhusuan.git to /tmp/pip-req-build-vcmn51ju
  Running command git clone -q https://github.com/thu-ml/zhusuan.git /tmp/pip-req-build-vcmn51ju
  Created wheel for zhusuan: filename=zhusuan-0.4.0-py2.py3-none-any.whl size=73591 sha256=08d32d74bbf8e14237eef49f76dd290241c28b461a4ec0f4a50187fe4cf0e62d
  Stored in directory: /tmp/pip-ephem-wheel-cache-dutu9u4z/wheels/45/cb/f5/bfd913ae94924c3151ac7d20dab61be39e90a2b07bdc6cb75e
Successfully built zhusuan
  Cloning https://github.com/haowen-xu/tfsnippet.git (to revision v0.2.0-alpha1) to /tmp/pip-req-build-55ax8ce9
  Running command git clone -q https://github.com/haowen-xu/tfsnippet.git /tmp/pip-req-build-55ax8ce9
  Running command git checkout -q 7b43abdbdd29f1914dbc11b961b5d45b9de76653
     |████████████████████████████████| 61kB 2.7MB/s 
  Created wheel for TFSnippet: filename=TFSnippet-0.2.0a1-cp36-none-any.whl size=207813 sha256=8e416ca1bc35e3687e9b7155b4d86275b12831a0c3b7543e05169dcae8be0299
  Stored in 

In [1]:
%pip install matplotlib==3.1.1
%pip install pandas==1.0.0
%pip install numpy==1.16.0

In [2]:
%pip install scipy==1.2.0

In [3]:
%pip install scikit_learn==0.20.2

In [4]:
%pip install tensorflow-gpu==1.12.0

In [5]:
%pip install tensorflow_probability==0.5.0

In [7]:
%pip install tqdm==4.48.2

     |████████████████████████████████| 71kB 1.8MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [8]:
%pip install imageio==2.4.1

In [9]:
%pip install fs==2.3.0

In [10]:
%pip install click==7.0

In [15]:
# -*- coding: utf-8 -*-
import tensorflow as tf
from tfsnippet import Distribution, Normal

class RecurrentDistribution(Distribution):
    """
    A multi-variable distribution integrated with recurrent structure.
    """

    @property
    def dtype(self):
        return self._dtype

    @property
    def is_continuous(self):
        return self._is_continuous

    @property
    def is_reparameterized(self):
        return self._is_reparameterized

    @property
    def value_shape(self):
        return self.normal.value_shape

    def get_value_shape(self):
        return self.normal.get_value_shape()

    @property
    def batch_shape(self):
        return self.normal.batch_shape

    def get_batch_shape(self):
        return self.normal.get_batch_shape()

    def sample_step(self, a, t):
        z_previous, mu_q_previous, std_q_previous = a
        noise_n, input_q_n = t
        input_q_n = tf.broadcast_to(input_q_n,
                                    [tf.shape(z_previous)[0], tf.shape(input_q_n)[0], input_q_n.shape[1]])
        input_q = tf.concat([input_q_n, z_previous], axis=-1)
        mu_q = self.mean_q_mlp(input_q, reuse=tf.AUTO_REUSE)  # n_sample * batch_size * z_dim

        std_q = self.std_q_mlp(input_q)  # n_sample * batch_size * z_dim

        temp = tf.einsum('ik,ijk->ijk', noise_n, std_q)  # n_sample * batch_size * z_dim
        mu_q = tf.broadcast_to(mu_q, tf.shape(temp))
        std_q = tf.broadcast_to(std_q, tf.shape(temp))
        z_n = temp + mu_q

        return z_n, mu_q, std_q

    # @global_reuse
    def log_prob_step(self, _, t):

        given_n, input_q_n = t
        if len(given_n.shape) > 2:
            input_q_n = tf.broadcast_to(input_q_n,
                                        [tf.shape(given_n)[0], tf.shape(input_q_n)[0], input_q_n.shape[1]])
        input_q = tf.concat([given_n, input_q_n], axis=-1)
        mu_q = self.mean_q_mlp(input_q, reuse=tf.AUTO_REUSE)

        std_q = self.std_q_mlp(input_q)
        logstd_q = tf.log(std_q)
        precision = tf.exp(-2 * logstd_q)
        if self._check_numerics:
            precision = tf.check_numerics(precision, "precision")
        log_prob_n = - 0.9189385332046727 - logstd_q - 0.5 * precision * tf.square(tf.minimum(tf.abs(given_n - mu_q),
                                                                                              1e8))
        return log_prob_n

    def __init__(self, input_q, mean_q_mlp, std_q_mlp, z_dim, window_length=100, is_reparameterized=True,
                 check_numerics=True):
        super(RecurrentDistribution, self).__init__()
        self.normal = Normal(mean=tf.zeros([window_length, z_dim]), std=tf.ones([window_length, z_dim]))
        self.std_q_mlp = std_q_mlp
        self.mean_q_mlp = mean_q_mlp
        self._check_numerics = check_numerics
        self.input_q = tf.transpose(input_q, [1, 0, 2])
        self._dtype = input_q.dtype
        self._is_reparameterized = is_reparameterized
        self._is_continuous = True
        self.z_dim = z_dim
        self.window_length = window_length
        self.time_first_shape = tf.convert_to_tensor([self.window_length, tf.shape(input_q)[0], self.z_dim])

    def sample(self, n_samples=1024, is_reparameterized=None, group_ndims=0, compute_density=False,
               name=None):

        from tfsnippet.stochastic import StochasticTensor
        if n_samples is None:
            n_samples = 1
            n_samples_is_none = True
        else:
            n_samples_is_none = False
        with tf.name_scope(name=name, default_name='sample'):
            noise = self.normal.sample(n_samples=n_samples)

            noise = tf.transpose(noise, [1, 0, 2])  # window_length * n_samples * z_dim
            noise = tf.truncated_normal(tf.shape(noise))

            time_indices_shape = tf.convert_to_tensor([n_samples, tf.shape(self.input_q)[1], self.z_dim])

            samples = tf.scan(fn=self.sample_step,
                              elems=(noise, self.input_q),
                              initializer=(tf.zeros(time_indices_shape),
                                           tf.zeros(time_indices_shape),
                                           tf.ones(time_indices_shape)),
                              back_prop=False
                              )[0]  # time_step * n_samples * batch_size * z_dim

            samples = tf.transpose(samples, [1, 2, 0, 3])  # n_samples * batch_size * time_step *  z_dim

            if n_samples_is_none:
                t = StochasticTensor(
                    distribution=self,
                    tensor=tf.reduce_mean(samples, axis=0),
                    n_samples=1,
                    group_ndims=group_ndims,
                    is_reparameterized=self.is_reparameterized
                )
            else:
                t = StochasticTensor(
                    distribution=self,
                    tensor=samples,
                    n_samples=n_samples,
                    group_ndims=group_ndims,
                    is_reparameterized=self.is_reparameterized
                )
            if compute_density:
                with tf.name_scope('compute_prob_and_log_prob'):
                    log_p = t.log_prob()
                    t._self_prob = tf.exp(log_p)
            return t

    def log_prob(self, given, group_ndims=0, name=None):
        with tf.name_scope(name=name, default_name='log_prob'):
            if len(given.shape) > 3:
                time_indices_shape = tf.convert_to_tensor([tf.shape(given)[0], tf.shape(self.input_q)[1], self.z_dim])
                given = tf.transpose(given, [2, 0, 1, 3])
            else:
                time_indices_shape = tf.convert_to_tensor([tf.shape(self.input_q)[1], self.z_dim])
                given = tf.transpose(given, [1, 0, 2])
            log_prob = tf.scan(fn=self.log_prob_step, elems=(given, self.input_q), initializer=tf.zeros(time_indices_shape), back_prop=False)
            if len(given.shape) > 3:
                log_prob = tf.transpose(log_prob, [1, 2, 0, 3])
            else:
                log_prob = tf.transpose(log_prob, [1, 0, 2])

            if group_ndims == 1:
                log_prob = tf.reduce_sum(log_prob, axis=-1)
            return log_prob

    def prob(self, given, group_ndims=0, name=None):
        with tf.name_scope(name=name, default_name='prob'):
            log_prob = self.log_prob(given, group_ndims, name)
            return tf.exp(log_prob)


ImportError: ignored

In [ ]:
# -*- coding: utf-8 -*-
import tensorflow as tf
from tfsnippet.bayes import BayesianNet
from tfsnippet.distributions import Distribution
from tfsnippet.stochastic import StochasticTensor, validate_n_samples_arg
from tfsnippet.utils import (instance_reuse, is_tensor_object, reopen_variable_scope, VarScopeObject)
from tfsnippet.variational import VariationalChain


class VAE(VarScopeObject):
    """
    A general implementation of variational auto-encoder as module.

    The variational auto-encoder ("Auto-Encoding Variational Bayes",
    Kingma, D.P. and Welling) is a deep Bayesian network, with observed
    variable `x` and latent variable `z`.  The generative process
    starts from `z` with prior distribution :math:`p(z)`, following a
    hidden network :math:`h(z)`, then comes to `x` with distribution
    :math:`p(x|h(z))`.  To do posterior inference of :math:`p(z|x)`,
    variational inference techniques are adopted, to train a separated
    distribution :math:`q(z|h(x))` (:math:`h(x)` denoting the hidden network)
    to approximate :math:`p(z|x)`.

    This class provides a general implementation of variational auto-encoder,
    with customizable :math:`p(z)`, :math:`p(x|h(z))`, :math:`q(z|h(x))`,
    as well as the hidden networks :math:`h(z)` and :math:`h(x)`.

    For example, to construct a VAE with diagonal Normal `z` and `x`:

    .. code-block:: python

        from tensorflow import keras as K
        from tfsnippet.modules import VAE, DictMapper, Sequential
        from tfsnippet.distributions import Normal

        batch_size = 128
        x_dims, z_dims = 100, 10
        vae = VAE(
            p_z=Normal(mean=tf.zeros([batch_size, z_dims]),
                       std=tf.ones([batch_size, x_dims])),
            p_x_given_z=Normal,
            q_z_given_x=Normal,
            h_for_p_x=Sequential([
                K.layers.Dense(100, activation=tf.nn.relu),
                DictMapper({'mean': K.layers.Dense(x_dims),
                            'logstd': K.layers.Dense(x_dims)})
            ]),
            h_for_q_z=Sequential([
                K.layers.Dense(100, activation=tf.nn.relu),
                DictMapper({'mean': K.layers.Dense(z_dims),
                            'logstd': K.layers.Dense(z_dims)})
            ])
        )

    To train the `vae`:

    .. code-block:: python

        # Automatically derive a single-sample loss.
        # Depending on ``z.is_reparameterized``, it might be derived by
        # `sgvb` (is_reparameterized == True) or `reinforce` (otherwise).
        loss = vae.get_training_loss(x)

        # Automatically derive a multi-sample loss.
        # Depending on ``z.is_reparameterized``, it might be derived by
        # `iwae` (is_reparameterized == True) or `vimco` (otherwise).
        loss = vae.get_training_loss(x, n_z=10)

        # Or manually derive a reweighted wake-sleep training loss.
        # Note the `VariationalTrainingObjectives` produce per-data
        # training objectives, instead of a 0-d scalar loss as the
        # `VAE.get_training_loss` does.
        chain = vae.chain(x, n_z=10)
        loss = tf.reduce_mean(chain.vi.training.rws_wake())

    To map from `x` to `z`:

    .. code-block:: python

        # use the :class:`Module` interface for one-to-one mapping
        z = vae(x)

        # use the :class:`Module` interface for multiple `z` samples
        z = vae(x, n_z=10)

        # or obtain the variational :class:`BayesianNet` with observed `z`
        q_net = vae.variational(x, z=observed_z)
        z_log_prob = q_net['z'].log_prob()

    To reconstruct `x`:

    .. code-block:: python

        # use the :meth:`VAE.reconstruct` for obtaining one `x` sample
        x_reconstructed = vae.reconstruct(x)

        # to obtain multiple `z` samples, and further multiple `x` samples
        # (this results in 100 `x` samples for each input `x`)
        x_reconstructed = vae.reconstruct(x, n_z=10, n_x=10)

    To sample `x` from prior `z` or observed `z`:

    .. code-block:: python

        # sample multiple prior `z`, then one `x` for each `z`
        x = vae.model(n_z=10)['x']

        # sample multiple `x` for each observed `z`
        x = vae.model(z=observed_z, n_x=10)['x']
    """

    def __init__(self, p_z, p_x_given_z, q_z_given_x, h_for_p_x, h_for_q_z,
                 z_group_ndims=1, x_group_ndims=1, is_reparameterized=None,
                 name=None, scope=None):
        """
        Construct the :class:`VAE`.

        Args:
            p_z (Distribution): :math:`p(z)`, the distribution instance.
            p_x_given_z: :math:`p(x|h(z))`, a distribution class or
                a :class:`DistributionFactory` object.
            q_z_given_x: :math:`q(z|h(x))`, a distribution class or
                a :class:`DistributionFactory` object.
            h_for_p_x (Module): :math:`h(z)`, the hidden network module for
                :math:`p(x|h(z))`. The output of `h_for_p_x` must be a
                ``dict[str, any]``, the parameters for `p_x_given_z`.
            h_for_q_z (Module): :math:`h(x)`, the hidden network module for
                :math:`q(z|h(x))`. The output of `h_for_q_z` must be a
                ``dict[str, any]``, the parameters for `q_z_given_x`.
            z_group_ndims (int or tf.Tensor): `group_ndims` for `z`. (default 1)
            x_group_ndims (int or tf.Tensor): `group_ndims` for `x`. (default 1)
            is_reparameterized (bool or None): Whether or not `z` should be
                re-parameterized? (default :obj:`None`, following the settings
                of z distributions.)
            name (str): Optional name of this module
                (argument of :class:`~tfsnippet.utils.VarScopeObject`).
            scope (str): Optional scope of this module
                (argument of :class:`~tfsnippet.utils.VarScopeObject`).

        See Also:
            :meth:`tfsnippet.distributions.Distribution.log_prob` for
                contents about `group_ndims`.
        """
        if not isinstance(p_z, Distribution):
            raise TypeError('`p_z` must be an instance of `Distribution`')
        if not callable(h_for_p_x):
            raise TypeError('`h_for_p_x` must be an instance of `Module` or '
                            'a callable object')
        if not callable(h_for_q_z):
            raise TypeError('`h_for_q_z` must be an instance of `Module` or '
                            'a callable object')
        super(VAE, self).__init__(name=name, scope=scope)

        # Defensive coding: wrap `h_for_p_x` and `h_for_q_z` in reused scope.
        if not isinstance(h_for_p_x, VarScopeObject):
            with reopen_variable_scope(self.variable_scope):
                h_for_p_x = Lambda(h_for_p_x, name='h_for_p_x')
        if not isinstance(h_for_q_z, VarScopeObject):
            with reopen_variable_scope(self.variable_scope):
                h_for_q_z = Lambda(h_for_q_z, name='h_for_q_z')

        self._p_z = p_z
        self._p_x_given_z = p_x_given_z
        self._q_z_given_x = q_z_given_x
        self._h_for_p_x = h_for_p_x
        self._h_for_q_z = h_for_q_z
        self._z_group_ndims = z_group_ndims
        self._x_group_ndims = x_group_ndims
        self._is_reparameterized = is_reparameterized

    def __call__(self, inputs, **kwargs):
        with reopen_variable_scope(self.variable_scope):
            # Here `reopen_name_scope` is set to True, so that multiple
            # calls to the same Module instance will always generate operations
            # within the original name scope.
            # However, in order for ``tf.variable_scope(default_name=...)``
            # to work properly with variable reusing, we must generate a nested
            # unique name scope.
            with tf.name_scope('forward'):
                return self._forward(inputs, **kwargs)

    @property
    def p_z(self):
        """
        Get :math:`p(z)`, the prior distribution of `z`.

        Returns:
            Distribution: The distribution instance.
        """
        return self._p_z

    @property
    def p_x_given_z(self):
        """
        Get the factory for :math:`p(x|h(z))`.

        Returns:
            DistributionFactory: The distribution factory.
        """
        return self._p_x_given_z

    @property
    def q_z_given_x(self):
        """
        Get the factory for :math:`q(z|h(x))`.

        Returns:
            DistributionFactory: The distribution factory.
        """
        return self._q_z_given_x

    @property
    def h_for_p_x(self):
        """
        Get :math:`h(z)`, the hidden network for :math:`p(x|h(z))`.

        Returns:
            Module: The hidden network.
        """
        return self._h_for_p_x

    @property
    def h_for_q_z(self):
        """
        Get :math:`h(x)`, the hidden network for :math:`q(z|h(x))`.

        Returns:
            Module: The hidden network.
        """
        return self._h_for_q_z

    @property
    def z_group_ndims(self):
        """Get the `group_ndims` for `z`."""
        return self._z_group_ndims

    @property
    def x_group_ndims(self):
        """Get the `group_ndims` for `x`."""
        return self._x_group_ndims

    @property
    def is_reparameterized(self):
        """Whether or not `z` is re-parameterized?"""
        return self._is_reparameterized

    @instance_reuse
    def variational(self, x, z=None, n_z=None, posterior_flow=None):
        """
        Derive an instance of :math:`q(z|h(x))`, the variational net.

        Args:
            x: The observation `x` for the variational net.
            z: If specified, observe `z` in the variational net.
                (default :obj:`None`)
            n_z: The number of `z` samples to take for each `x`, if `z`
                is not observed. (default :obj:`None`, one sample for
                each `x`, without dedicated sampling dimension)

                It is recommended to specify this argument even if `z`
                is observed, to make explicit how many samples are there
                in the observation.

        Returns:
            BayesianNet: The variational net.
        """
        observed = {}
        if z is not None:
            observed['z'] = z
        net = BayesianNet(observed=observed)
        with tf.variable_scope('h_for_q_z'):
            z_params = self.h_for_q_z(x)
        with tf.variable_scope('q_z_given_x'):
            q_z_given_x = self.q_z_given_x(**z_params)
            assert (isinstance(q_z_given_x, Distribution))
        with tf.name_scope('z'):
            z = net.add('z', q_z_given_x, n_samples=n_z,
                        group_ndims=self.z_group_ndims,
                        is_reparameterized=self.is_reparameterized,
                        flow=posterior_flow)
        return net

    @instance_reuse
    def model(self, z=None, x=None, n_z=None, n_x=None):
        """
        Derive an instance of :math:`p(x|h(z))`, the model net.

        Args:
            z: If specified, observe `z` in the model net. (default :obj:`None`)
            x: If specified, observe `x` in the model net. (default :obj:`None`)
            n_z: The number of `z` samples to take for each `x`, if `z`
                is not observed. (default :obj:`None`, one `z` sample for
                each `x`, without dedicated sampling dimension)

                It is recommended to specify this argument even if `z`
                is observed, to make explicit how many samples are there
                in the observation.
            n_x: The number of `x` samples to take for each `z`, if `x`
                is not observed. (default :obj:`None`, one `x` sample for
                each `z`, without dedicated sampling dimension)

                It is recommended to specify this argument even if `x`
                is observed, to make explicit how many samples are there
                in the observation.

        Returns:
            BayesianNet: The variational net.
        """
        observed = {k: v for k, v in [('z', z), ('x', x)] if v is not None}
        net = BayesianNet(observed=observed)
        with tf.name_scope('z'):
            z = net.add('z', self.p_z, n_samples=n_z,
                        group_ndims=self.z_group_ndims,
                        is_reparameterized=self.is_reparameterized)
        with tf.variable_scope('h_for_p_x'):
            x_params = self.h_for_p_x(z)
        with tf.variable_scope('p_x_given_z'):
            p_x_given_z = self.p_x_given_z(**x_params)
            assert (isinstance(p_x_given_z, Distribution))
        with tf.name_scope('x'):
            x = net.add('x', p_x_given_z, n_samples=n_x,
                        group_ndims=self.x_group_ndims)
        return net

    def chain(self, x, n_z=None, posterior_flow=None):
        """
        Chain :math:`q(z|h(x))` and :math:`p(x,z|h(x))` together.

        This method chains the variational net :math:`q(z|h(x))` and the
        model net :math:`p(x,z|h(x))` together, with specified observation
        `x`.  It is typically used to derive the training objectives of VAE.
        It can also be used to calculate the `reconstruction probability`
        ("Variational Autoencoder based Anomaly Detection using Reconstruction
        Probability", An, J. and Cho, S. 2015) of `x`.

        Notes:
            The constructed :class:`~tfsnippet.variational.VariationalChain`
            have `x` observed in its `model` net, thus this method cannot
            be used to get reconstructed samples.  Use :meth:`reconstruct`
            instead to obtain `x` samples.

        Args:
            x: The input observation `x`.
            n_z: Number of `z` samples to take. (default :obj:`None`)

        Returns:
            VariationalChain: The variational chain.
        """
        with tf.name_scope('VAE.chain'):
            q_net = self.variational(x, n_z=n_z, posterior_flow=posterior_flow)

            # automatically detect the `latent_axis` for this chain
            if n_z is not None:
                latent_axis = 0
            else:
                latent_axis = None

            chain = q_net.variational_chain(
                lambda observed: self.model(n_z=n_z, n_x=None, **observed),
                latent_axis=latent_axis,
                observed={'x': x}
            )
        return chain

    def get_training_loss(self, x, n_z=None):
        """
        Get the training loss for this VAE.

        The variational solver is automatically chosen according to
        `z.is_reparameterized`, and the argument `n_z`, by the following rules:

        1. If `z.is_reparameterized` is :obj:`True`, then:

            1. If `n_z` > 1, use `iwae`.
            2. If `n_z` == 1 or `n_z` is :obj:`None`, use `sgvb`.

        2. If `z.is_reparameterized` is :obj:`False`, then:

            1. If `n_z` > 1, use `vimco`.
            2. If `n_z` == 1 or `n_z` is :obj:`None`, use `reinforce`.

        Dynamic `n_z` is not supported by this method.  Also, Reweighted
        Wake-Sleep algorithm is not a choice of this method.  To derive
        the training loss for either situation, use :meth:`chain`
        to obtain a :class:`~tfsnippet.variational.VariationalChain`,
        and further obtain the loss by `chain.vi.training.[algorithm]`.

        Args:
            x: The input observation `x`.
            n_z (int or None): Number of `z` samples to take.  Must be
                :obj:`None` or a constant integer.  Dynamic tensors are not
                accepted, since we cannot automatically choose a variational
                solver for undeterministic `n_z`. (default :obj:`None`)

        Returns:
            tf.Tensor: A 0-d tensor, the training loss which can be optimized
                by gradient descent.

        See Also:
            :class:`tfsnippet.variational.VariationalChain`,
            :class:`tfsnippet.variational.VariationalTrainingObjectives`
        """
        with tf.name_scope('VAE.get_training_loss'):
            if n_z is not None:
                if is_tensor_object(n_z):
                    raise TypeError('Cannot choose the variational solver '
                                    'automatically for dynamic `n_z`')
                n_z = validate_n_samples_arg(n_z, 'n_z')

            # derive the variational chain
            chain = self.chain(x, n_z)
            z = chain.variational['z']

            # auto choose a variational solver for training loss
            if n_z is not None and n_z > 1:
                if z.is_reparameterized:
                    solver = chain.vi.training.iwae
                else:
                    solver = chain.vi.training.vimco
            else:
                if z.is_reparameterized:
                    solver = chain.vi.training.sgvb
                else:
                    solver = chain.vi.training.reinforce

            # derive the training loss
            return tf.reduce_mean(solver())

    def reconstruct(self, x, n_z=None, n_x=None, posterior_flow=None):
        """
        Sample reconstructed `x` from :math:`p(x|h(z))`, where `z` is (are)
        sampled from :math:`q(z|h(x))` using the specified observation `x`.

        Args:
            x: The observation `x` for :math:`q(z|h(x))`.
            n_z: Number of intermediate `z` samples to take for each input `x`.
            n_x: Number of reconstructed `x` samples to take for each `z`.

        Returns:
            StochasticTensor: The reconstructed samples `x`.
        """
        with tf.name_scope('VAE.reconstruct'):
            q_net = self.variational(x, n_z=n_z, posterior_flow=posterior_flow)
            model = self.model(z=q_net['z'], n_z=n_z, n_x=n_x)
            return model['x']

    def _forward(self, inputs, n_z=None, **kwargs):
        """
        Get a `z` sample from :math:`q(z|h(x))`, using the variational net.

        Args:
            inputs: The input `x`.
            n_z: Number of samples to taken for `z`. (default :obj:`None`)
            \**kwargs: Capturing and ignoring all other parameters.  This is
                the default behavior of a :class:`Module`.

        Returns:
            StochasticTensor: The `z` samples.
        """
        q_net = self.variational(inputs, z=None, n_z=n_z, **kwargs)
        return q_net['z']


class Lambda(VarScopeObject):
    """
    Wrapping arbitrary function into a neural network :class:`Module`.

    This class wraps an arbitrary function or lambda expression into
    a neural network :class:`Module`, reusing the variables created
    within the specified function.

    For example, one may wrap :func:`tensorflow.contrib.layers.fully_connected`
    into a reusable module with :class:`Lambda` component as follows:

    .. code-block:: python

        import functools
        from tensorflow.contrib import layers

        dense = Lambda(
            functools.partial(
                layers.fully_connected,
                num_outputs=100,
                activation_fn=tf.nn.relu
            )
        )
    """

    def __init__(self, f, name=None, scope=None):
        """
        Construct the :class:`Lambda`.

        Args:
            f ((inputs, \**kwargs) -> outputs): The function or lambda
                expression which derives the outputs.
            name (str): Optional name of this module
                (argument of :class:`~tfsnippet.utils.VarScopeObject`).
            scope (str): Optional scope of this module
                (argument of :class:`~tfsnippet.utils.VarScopeObject`).
        """
        super(Lambda, self).__init__(name=name, scope=scope)
        self._factory = f

    def _forward(self, inputs, **kwargs):
        return self._factory(inputs, **kwargs)

    def __call__(self, inputs, **kwargs):
        with reopen_variable_scope(self.variable_scope):
            # Here `reopen_name_scope` is set to True, so that multiple
            # calls to the same Module instance will always generate operations
            # within the original name scope.
            # However, in order for ``tf.variable_scope(default_name=...)``
            # to work properly with variable reusing, we must generate a nested
            # unique name scope.
            with tf.name_scope('forward'):
                return self._forward(inputs, **kwargs)



In [13]:
# -*- coding: utf-8 -*-
import logging

import tensorflow as tf
import tensorflow_probability as tfp
from tfsnippet.distributions import Distribution


class TfpDistribution(Distribution):
    """
    A wrapper class for `tfp.distributions.Distribution`
    """

    @property
    def is_continuous(self):
        return self._is_continuous

    def __init__(self, distribution):
        if not isinstance(distribution, tfp.distributions.Distribution):
            raise TypeError('`distribution` is not an instance of `tfp.'
                            'distributions.Distribution`')
        super(TfpDistribution, self).__init__()
        self._distribution = distribution
        self._is_continuous = True
        self._is_reparameterized = self._distribution.reparameterization_type is tfp.distributions.FULLY_REPARAMETERIZED

    def __repr__(self):
        return 'Distribution({!r})'.format(self._distribution)

    @property
    def dtype(self):
        return self._distribution.dtype

    @property
    def is_reparameterized(self):
        return self._is_reparameterized

    @property
    def value_shape(self):
        return self._distribution.event_shape

    def get_value_shape(self):
        return self._distribution.event_shape

    @property
    def batch_shape(self):
        return self._distribution.batch_shape

    def get_batch_shape(self):
        return self._distribution.batch_shape()

    def sample(self, n_samples=None, is_reparameterized=None, group_ndims=0, compute_density=False,
               name=None):
        from tfsnippet.stochastic import StochasticTensor
        if n_samples is None or n_samples < 2:
            n_samples = 2
        with tf.name_scope(name=name, default_name='sample'):
            samples = self._distribution.sample(n_samples)
            samples = tf.reduce_mean(samples, axis=0)
            t = StochasticTensor(
                distribution=self,
                tensor=samples,
                n_samples=n_samples,
                group_ndims=group_ndims,
                is_reparameterized=self.is_reparameterized
            )
            if compute_density:
                with tf.name_scope('compute_prob_and_log_prob'):
                    log_p = t.log_prob()
                    t._self_prob = tf.exp(log_p)
            return t

    def log_prob(self, given, group_ndims=0, name=None):
        with tf.name_scope(name=name, default_name='log_prob'):
            log_prob, _, _, _, _, _, _ = self._distribution.forward_filter(given)
            return log_prob


def softplus_std(inputs, units, epsilon, name):
    return tf.nn.softplus(tf.layers.dense(inputs, units, name=name, reuse=tf.AUTO_REUSE)) + epsilon


def rnn(x,
        window_length,
        rnn_num_hidden,
        rnn_cell='GRU',
        hidden_dense=2,
        dense_dim=200,
        time_axis=1,
        name='rnn'):
    from tensorflow.contrib import rnn
    with tf.variable_scope(name, reuse=tf.AUTO_REUSE):
        if len(x.shape) == 4:
            x = tf.reduce_mean(x, axis=0)
        elif len(x.shape) != 3:
            logging.error("rnn input shape error")
        x = tf.unstack(x, window_length, time_axis)

        if rnn_cell == 'LSTM':
            # Define lstm cells with TensorFlow
            # Forward direction cell
            fw_cell = rnn.BasicLSTMCell(rnn_num_hidden,
                                        forget_bias=1.0)
        elif rnn_cell == "GRU":
            fw_cell = tf.nn.rnn_cell.GRUCell(rnn_num_hidden)
        elif rnn_cell == 'Basic':
            fw_cell = tf.nn.rnn_cell.BasicRNNCell(rnn_num_hidden)
        else:
            raise ValueError("rnn_cell must be LSTM or GRU")

        # Get lstm cell output

        try:
            outputs, _ = rnn.static_rnn(fw_cell, x, dtype=tf.float32)
        except Exception:  # Old TensorFlow version only returns outputs not states
            outputs = rnn.static_rnn(fw_cell, x, dtype=tf.float32)
        outputs = tf.stack(outputs, axis=time_axis)
        for i in range(hidden_dense):
            outputs = tf.layers.dense(outputs, dense_dim)
        return outputs
    # return size: (batch_size, window_length, rnn_num_hidden)


def wrap_params_net(inputs, h_for_dist, mean_layer, std_layer):
    with tf.variable_scope('hidden', reuse=tf.AUTO_REUSE):
        h = h_for_dist(inputs)
    return {
        'mean': mean_layer(h),
        'std': std_layer(h),
    }


def wrap_params_net_srnn(inputs, h_for_dist):
    with tf.variable_scope('hidden', reuse=tf.AUTO_REUSE):
        h = h_for_dist(inputs)
    return {
        'input_q': h
    }


ImportError: ignored

In [14]:
# -*- coding: utf-8 -*-
from functools import partial
import tensorflow as tf
import tfsnippet as spt
from tensorflow.python.ops.linalg.linear_operator_identity import LinearOperatorIdentity
from tensorflow_probability.python.distributions import LinearGaussianStateSpaceModel, MultivariateNormalDiag
from tfsnippet.distributions import Normal
from tfsnippet.utils import VarScopeObject, reopen_variable_scope
from tfsnippet.variational import VariationalInference

from omni_anomaly.recurrent_distribution import RecurrentDistribution
from omni_anomaly.vae import Lambda, VAE
from omni_anomaly.wrapper import TfpDistribution, softplus_std, rnn, wrap_params_net


class OmniAnomaly(VarScopeObject):
    def __init__(self, config, name=None, scope=None):
        self.config = config
        super(OmniAnomaly, self).__init__(name=name, scope=scope)
        with reopen_variable_scope(self.variable_scope):
            if config.posterior_flow_type == 'nf':
                self._posterior_flow = spt.layers.planar_normalizing_flows(
                    config.nf_layers, name='posterior_flow')
            else:
                self._posterior_flow = None
            self._window_length = config.window_length
            self._x_dims = config.x_dim
            self._z_dims = config.z_dim
            self._vae = VAE(
                p_z=TfpDistribution(
                    LinearGaussianStateSpaceModel(
                        num_timesteps=config.window_length,
                        transition_matrix=LinearOperatorIdentity(config.z_dim),
                        transition_noise=MultivariateNormalDiag(
                            scale_diag=tf.ones([config.z_dim])),
                        observation_matrix=LinearOperatorIdentity(config.z_dim),
                        observation_noise=MultivariateNormalDiag(
                            scale_diag=tf.ones([config.z_dim])),
                        initial_state_prior=MultivariateNormalDiag(
                            scale_diag=tf.ones([config.z_dim]))
                    )
                ) if config.use_connected_z_p else Normal(mean=tf.zeros([config.z_dim]), std=tf.ones([config.z_dim])),
                p_x_given_z=Normal,
                q_z_given_x=partial(RecurrentDistribution,
                                    mean_q_mlp=partial(tf.layers.dense, units=config.z_dim, name='z_mean', reuse=tf.AUTO_REUSE),
                                    std_q_mlp=partial(softplus_std, units=config.z_dim, epsilon=config.std_epsilon,
                                                      name='z_std'),
                                    z_dim=config.z_dim, window_length=config.window_length) if config.use_connected_z_q else Normal,
                h_for_p_x=Lambda(
                    partial(
                        wrap_params_net,
                        h_for_dist=lambda x: rnn(x=x,
                                                 window_length=config.window_length,
                                                 rnn_num_hidden=config.rnn_num_hidden,
                                                 hidden_dense=2,
                                                 dense_dim=config.dense_dim,
                                                 name='rnn_p_x'),
                        mean_layer=partial(
                            tf.layers.dense, units=config.x_dim, name='x_mean', reuse=tf.AUTO_REUSE
                        ),
                        std_layer=partial(
                            softplus_std, units=config.x_dim, epsilon=config.std_epsilon,
                            name='x_std'
                        )
                    ),
                    name='p_x_given_z'
                ),
                h_for_q_z=Lambda(
                    lambda x: {'input_q': rnn(x=x,
                                              window_length=config.window_length,
                                              rnn_num_hidden=config.rnn_num_hidden,
                                              hidden_dense=2,
                                              dense_dim=config.dense_dim,
                                              name="rnn_q_z")},
                    name='q_z_given_x'
                ) if config.use_connected_z_q else Lambda(
                    partial(
                        wrap_params_net,
                        h_for_dist=lambda x: rnn(x=x,
                                                 window_length=config.window_length,
                                                 rnn_num_hidden=config.rnn_num_hidden,
                                                 hidden_dense=2,
                                                 dense_dim=config.dense_dim,
                                                 name="rnn_q_z"),
                        mean_layer=partial(
                            tf.layers.dense, units=config.z_dim, name='z_mean', reuse=tf.AUTO_REUSE
                        ),
                        std_layer=partial(
                            softplus_std, units=config.z_dim, epsilon=config.std_epsilon,
                            name='z_std'
                        )
                    ),
                    name='q_z_given_x'
                )
            )

    @property
    def x_dims(self):
        """Get the number of `x` dimensions."""
        return self._x_dims

    @property
    def z_dims(self):
        """Get the number of `z` dimensions."""
        return self._z_dims

    @property
    def vae(self):
        """
        Get the VAE object of this :class:`OmniAnomaly` model.

        Returns:
            VAE: The VAE object of this model.
        """
        return self._vae

    @property
    def window_length(self):
        return self._window_length

    def get_training_loss(self, x, n_z=None):
        """
        Get the training loss for `x`.

        Args:
            x (tf.Tensor): 2-D `float32` :class:`tf.Tensor`, the windows of
                KPI observations in a mini-batch.
            n_z (int or None): Number of `z` samples to take for each `x`.
                (default :obj:`None`, one sample without explicit sampling
                dimension)

        Returns:
            tf.Tensor: 0-d tensor, the training loss, which can be optimized
                by gradient descent algorithms.
        """
        with tf.name_scope('training_loss'):
            chain = self.vae.chain(x, n_z=n_z, posterior_flow=self._posterior_flow)
            x_log_prob = chain.model['x'].log_prob(group_ndims=0)
            log_joint = tf.reduce_sum(x_log_prob, -1)
            chain.vi.training.sgvb()
            vi = VariationalInference(
                log_joint=log_joint,
                latent_log_probs=chain.vi.latent_log_probs,
                axis=chain.vi.axis
            )
            loss = tf.reduce_mean(vi.training.sgvb())
            return loss

    def get_score(self, x, n_z=None,
                  last_point_only=True):
        """
        Get the reconstruction probability for `x`.

        The larger `reconstruction probability`, the less likely a point
        is anomaly.  You may take the negative of the score, if you want
        something to directly indicate the severity of anomaly.

        Args:
            x (tf.Tensor): 2-D `float32` :class:`tf.Tensor`, the windows of
                KPI observations in a mini-batch.
            n_z (int or None): Number of `z` samples to take for each `x`.
                (default :obj:`None`, one sample without explicit sampling
                dimension)
            last_point_only (bool): Whether to obtain the reconstruction
                probability of only the last point in each window?
                (default :obj:`True`)

        Returns:
            tf.Tensor: The reconstruction probability, with the shape
                ``(len(x) - self.x_dims + 1,)`` if `last_point_only` is
                :obj:`True`, or ``(len(x) - self.x_dims + 1, self.x_dims)``
                if `last_point_only` is :obj:`False`.  This is because the
                first ``self.x_dims - 1`` points are not the last point of
                any window.
        """
        with tf.name_scope('get_score'):
            x_r = x

            # get the reconstruction probability
            print('-' * 30, 'testing', '-' * 30)
            q_net = self.vae.variational(x=x_r, n_z=n_z, posterior_flow=self._posterior_flow)  # notice: x=x_r
            p_net = self.vae.model(z=q_net['z'], x=x, n_z=n_z)  # notice: x=x
            z_samples = q_net['z'].tensor
            z_mean = tf.reduce_mean(z_samples, axis=0) if n_z is not None else z_samples
            z_std = tf.sqrt(tf.reduce_sum(tf.square(z_samples - z_mean), axis=0) / (n_z - 1)) \
                if n_z is not None and n_z > 1 else tf.zeros_like(z_mean)
            z = tf.concat((z_mean, z_std), axis=-1)

            r_prob = p_net['x'].log_prob(group_ndims=int(not self.config.get_score_on_dim))

            if last_point_only:
                r_prob = r_prob[:, -1]
            return r_prob, z


ImportError: ignored

In [ ]:
? pip